#This Google Colab page describes how to use trained machine learning bedload model described in the paper "Development of a Machine Leanring Model for River Bedload" by Hosseiny et al. (2022)

**Installing required packages as needed**

In [ ]:
"""
Created on Saturday September 24 2022
@author: Hossein Hosseiny
"""

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras



In [ ]:
# Connecting the Colab to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Define or Load the Data.** 

The data can be imported by Pandas library. For instance and in case of an Excel file:

data = pd.read_excel(data.xlsx)

It should be noted that the order of the variables must be as

[ Width (m),	Bed Slope (m/m), Discharge (m$^3$/s), D16 (m), D50 (m), D84 (m), D90 (m) ]

To be compatible with the pretrianed model. 

It should be also noted that the data (using for prediction) for all variables should be between the range of the data used for training the model which are presented in the original study.

Following is an example for running the model on a data batch of 3, representing three different rivers.


In [ ]:
data = pd.DataFrame([[6.68,	0.0026,	1.649	, 0.00268,	0.01,	0.021, 0.0273], 
                     [5.55, 0.0095, 0.58, 0.00042, 0.0024,0.02138, 0.03235],
                     [12.76, 0.0091, 9.6, 0.02503, 0.116, 0.247, 0.34]], 
                    columns=['W', 'S', 'Q', 'D16','D50','D84', 'D90'])

Each row in data shows an instance which consists of 7 variables (width, slope, discharge, D16, D50, D84, D90) used as input to the model.

In [ ]:
data

,W,S,Q,D16,D50,D84,D90
0,6.68,0.0026,1.649,0.00268,0.0100,0.02100,0.02730
1,5.55,0.0095,0.580,0.00042,0.0024,0.02138,0.03235
2,12.76,0.0091,9.600,0.02503,0.1160,0.24700,0.34000


**Normalizing the Data**

The followin steps scale (sc) data to [0-1] by min-max method after logarithmic transformation (base 10) is applied on data. 

The min-max values for all variables are obtained from the the original study. 

The base10 logarithm (L) of the **min** and **max** values of the data in the original study was estimated for all variables. For instance, such values for the river width was obtianed by

$w_{maxL} = log(max(Width))$

$w_{minL} = log(min(Width))$

Then the width data can be scaled as:

$W_{sc} = (log(W) -  w_{minL})/(w_{maxL} - w_{minL})$


In [ ]:
# Constant values here are associated with the logarithm of the minimum and maximum 
# of each variable used in the original study

data['W_sc'] = (np.log10(data['W']) -  -0.5228787452803376)/(3.008600171761918)
data['S_sc'] = (np.log10(data['S']) -  -4.045757490560675)/(3.179296398930892)
data['Q_sc'] = (np.log10(data['Q']) -  -4.301029995663981)/(6.932047378606766)
data['D16_sc'] = (np.log10(data['D16']) -  -3.8728952016351923)/(2.864121277327687)
data['D50_sc'] = (np.log10(data['D50']) -  -3.5228787452803374)/(2.862330186586778)
data['D84_sc'] = (np.log10(data['D84']) -  -3.3979400086720375)/(3.1327398382608846)
data['D90_sc'] = (np.log10(data['D90']) -  -2.9244530386074694)/(2.8144938134579296)


In [ ]:
data

,W,S,Q,D16,D50,D84,D90,W_sc,S_sc,Q_sc,D16_sc,D50_sc,D84_sc,D90_sc
0,6.68,0.0026,1.649,0.00268,0.0100,0.02100,0.02730,0.447934,0.459451,0.651792,0.454251,0.532042,0.549091,0.483432
1,5.55,0.0095,0.580,0.00042,0.0024,0.02138,0.03235,0.421183,0.636456,0.586329,0.173227,0.315509,0.551577,0.509622
2,12.76,0.0091,9.600,0.02503,0.1160,0.24700,0.34000,0.541358,0.630579,0.762156,0.793038,0.903927,0.890798,0.872602


#Bedload Prediction


**Load Pre-trained model**

May require adjusting the path to where the model is saved in

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Sed_ML/Model_Data/BEDLOAD_MODEL_FINAL') 

In [ ]:
# model prediction
qs_sc = model.predict(data[['W_sc','S_sc','Q_sc', 'D16_sc',	'D50_sc',	'D84_sc',	'D90_sc']])

In [ ]:
print(" Scaled & Log-transformed Bedloads \n", qs_sc )

 Scaled & Log-transformed Bedloads 
 [[0.68983376]
 [0.65461224]
 [0.29142514]]


**Scaling Back the Prediction**

The direct output of the model-similar to the inputs- is log-transformed varying between [0-1]. 

To get the final prediction, it is required to scale back the output of the model as shown below:

In [ ]:
# Constant values here are associated with the logarithm of the minimum and maximum 
# of bedlod data used in the original study 

log10_qs = qs_sc * (3.5697206171332327) + -0.9665762445130504
print("Predicted Log-transformed Bedloads \n", log10_qs)

Predicted Log-transformed Bedloads 
 [[1.4959375 ]
 [1.3702065 ]
 [0.07373011]]


In [ ]:
qs = 10 ** log10_qs
print('qs prediction(g/s/m)\n',qs )

qs prediction(g/s/m)
 [[31.328346 ]
 [23.453436 ]
 [ 1.1850321]]


In [ ]:
data['qs-g/s/m'] = qs

In [ ]:
data

,W,S,Q,D16,D50,D84,D90,W_sc,S_sc,Q_sc,D16_sc,D50_sc,D84_sc,D90_sc,qs-g/s/m
0,6.68,0.0026,1.649,0.00268,0.0100,0.02100,0.02730,0.447934,0.459451,0.651792,0.454251,0.532042,0.549091,0.483432,31.328346
1,5.55,0.0095,0.580,0.00042,0.0024,0.02138,0.03235,0.421183,0.636456,0.586329,0.173227,0.315509,0.551577,0.509622,23.453436
2,12.76,0.0091,9.600,0.02503,0.1160,0.24700,0.34000,0.541358,0.630579,0.762156,0.793038,0.903927,0.890798,0.872602,1.185032
